Importing dependencies

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, min
from pyspark.sql.window import Window
#import numpy as np
from pyspark.ml.recommendation import ALS


Creating Spark Session

In [2]:
def spark_session():
    spark = SparkSession \
        .builder \
        .appName("PySpark SQL") \
        .config("spark.executor.memory", "6g") \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()
    return spark
    
spark = spark_session()

your 131072x1 screen size is bogus. expect trouble
23/11/02 22:42:27 WARN Utils: Your hostname, FlyingPig resolves to a loopback address: 127.0.1.1; using 172.17.190.229 instead (on interface eth0)
23/11/02 22:42:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/02 22:42:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Reading data in PySpark

In [3]:
def read_tab(path):
    tab = spark.read.csv(path, header=True)
    return tab
"""
df_genome_scores = read_tab("../data/genome-scores.csv")
print("df_genome_scores table:")
df_genome_scores.show()
df_genome_tags = read_tab("../data/genome-tags.csv")
print("df_genome_tags table:")
df_genome_tags.show()
df_links = read_tab("../data/links.csv")
print("df_links table:")
df_links.show()
"""
df_movies = read_tab("../data/movies.csv")
print("df_movies table:")
df_movies.show()
df_ratings = read_tab("../data/ratings.csv")
print("df_ratings table:")
df_ratings.show()
"""
df_tags = read_tab("../data/tags.csv")
print("df_tags table:")
df_tags.show()
"""


df_movies table:
+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Dr

'\ndf_tags = read_tab("../data/tags.csv")\nprint("df_tags table:")\ndf_tags.show()\n'

Building content based model on the base of explicit ratings:
Stage 1: Creating 1 table with relevant for content based model data.



In [4]:
def prepare_data(df_movies, df_ratings):
    df_movies.createOrReplaceTempView("movies")
    df_ratings.createOrReplaceTempView("ratings")
    df_cbm_data = spark.sql("SELECT movies.movieId AS movieId_movie, movies.title, ratings.movieId AS movieId_rating, ratings.userId, ratings.rating FROM movies INNER JOIN ratings ON movies.movieId = ratings.movieId")
    df_cbm_data = df_cbm_data.drop("movieId_movie")
    df_cbm_data = df_cbm_data.withColumn("movieId_rating", col("movieId_rating").cast('int')) \
        .withColumn("userId", col("userId").cast('int')) \
        .withColumn("rating", col("rating").cast('int'))
    window_spec = Window.partitionBy("userId")
    df_cbm_data = df_cbm_data.withColumn("user_count", count("userId").over(window_spec))
    df_cbm_data = df_cbm_data.filter(col("user_count") >= 20)
    return df_cbm_data

df_cbm_data = prepare_data(df_movies, df_ratings)

df_cbm_data.show()
df_cbm_data.printSchema()
df_cbm_data.count()



+--------------------+--------------+------+------+----------+
|               title|movieId_rating|userId|rating|user_count|
+--------------------+--------------+------+------+----------+
|Twelve Monkeys (a...|            32|    28|     5|        49|
|Interview with th...|           253|    28|     5|        49|
|Star Wars: Episod...|           260|    28|     5|        49|
|     Outbreak (1995)|           292|    28|     3|        49|
|     Stargate (1994)|           316|    28|     5|        49|
|    True Lies (1994)|           380|    28|     4|        49|
| Blade Runner (1982)|           541|    28|     5|        49|
|  Heavy Metal (1981)|           610|    28|     5|        49|
|Mystery Science T...|           671|    28|     5|        49|
|Ghost in the Shel...|           741|    28|     5|        49|
|Dr. Strangelove o...|           750|    28|     5|        49|
|2001: A Space Ody...|           924|    28|     5|        49|
|E.T. the Extra-Te...|          1097|    28|     5|    

25000095

Building content based model on the base of explicit ratings:
Stage 2: Transforming table to make it ready for content based model.

In [5]:
def transform_data(df_cbm_data):
    df_matrix = df_cbm_data.select(col("userId"), col("movieId_rating").alias("movieId"), col("rating"))
    return df_matrix

df_matrix = transform_data(df_cbm_data)
df_matrix.show()
df_matrix.printSchema()
df_matrix.count()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    28|     32|     5|
|    28|    253|     5|
|    28|    260|     5|
|    28|    292|     3|
|    28|    316|     5|
|    28|    380|     4|
|    28|    541|     5|
|    28|    610|     5|
|    28|    671|     5|
|    28|    741|     5|
|    28|    750|     5|
|    28|    924|     5|
|    28|   1097|     5|
|    28|   1127|     5|
|    28|   1196|     5|
|    28|   1200|     5|
|    28|   1206|     5|
|    28|   1210|     5|
|    28|   1214|     5|
|    28|   1215|     5|
+------+-------+------+
only showing top 20 rows

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: integer (nullable = true)



25000095

Building content based model on the base of explicit ratings:
Stage 3: splitting data into train and test sets.

In [6]:
def split_data(df):
    train, test = df.randomSplit([0.8, 0.2])
    return train, test

df_train, df_test = split_data(df_matrix)

Building content based model on the base of explicit ratings:
Stage 4: Building ALS model.

In [7]:
def model_1(training_data, test_data):
    als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",
            rank=25, maxIter=100, regParam=.05, nonnegative=True,
            coldStartStrategy="drop", implicitPrefs=False)
    model = als.fit(training_data)
    predictions = model.transform(test_data)
    return predictions, model

predictions, model = model_1(df_train, df_test)

23/11/02 22:44:20 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/02 22:46:04 ERROR Executor: Exception in task 6.0 in stage 78.0 (TID 500)]
java.lang.StackOverflowError
	at java.base/java.io.ObjectInputStream$PeekInputStream.peek(ObjectInputStream.java:2871)
	at java.base/java.io.ObjectInputStream$BlockDataInputStream.peek(ObjectInputStream.java:3198)
	at java.base/java.io.ObjectInputStream$BlockDataInputStream.peekByte(ObjectInputStream.java:3208)
	at java.base/java.io.ObjectInputStream.readClassDesc(ObjectInputStream.java:1855)
	at java.base/java.io.ObjectInputStream.readOrdinaryObject(ObjectInputStream.java:2201)
	at java.base/java.io.ObjectInputStream.readObject0(ObjectInputStream.java:1687)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:489)
	at java.base/java.io.ObjectInputStream.readObject(ObjectInputStream.java:447)
	at scala.collection.immutable.List$SerializationProxy.readObject(List.scala:527)
	at jdk

Py4JError: An error occurred while calling o147.toString

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/flyingpig/codes/becode_projects/movie_analisys/env/lib/python3.10/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/flyingpig/codes/becode_projects/movie_analisys/env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/flyingpig/codes/becode_projects/movie_analisys/env/lib/python3.10/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
